## NER - Natasha

Natasha  - питоновская библиотека для извлечения именованных сущностей. Она похоже на Tomita-parser, но в ней все на чистом питоне, с открытым кодом и активно развивается. 

Если быть точнее, то natasha - набор готовых правил для парсера yargy. 

Есть например готовые правила для извлечения персон.

In [1]:
from natasha import NamesExtractor
from yargy import Parser, rule, or_, and_
from yargy.predicates import gram, is_capitalized, dictionary
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact
from IPython.display import display

In [2]:
sents = open('sents.txt', 'r', encoding='utf-8').read().splitlines()

Посмотрев на данные, я заметил, что часто имена (и должности) персон опускаются, тогда как фамилии присутствуют всегда.  
Тогда добавим это в правила, сделав факты `POSITION` и `first_name` опциональными.

In [11]:
Person = fact(
    'Person',
    ['position', 'name']
)

Name = fact(
    'Name',
    ['first', 'last']
)

POSITION = morph_pipeline([
    'премьер министр',
    'президент'
])

NAME = rule(
    gram('Name').interpretation(
        Name.first.inflected()
    ).optional(),
    gram('Surn').interpretation(
        Name.last.inflected()
    )
).interpretation(
    Name
)

PERSON = rule(
    POSITION.interpretation(
        Person.position.inflected()
    ).optional(),
    NAME.interpretation(
        Person.name
    )
).interpretation(
    Person
)


parser = Parser(PERSON)

Посмотрим, что получается:

In [12]:
matches = list()
for sent in sents:
    matches.extend(parser.findall(sent))
    
print(f'number of matches: {len(matches)}')

number of matches: 115


Столь простой поправкой удалось выделить достаточные 115 персон.

In [13]:
for m in matches[:10]: display(m.fact)

Person(position=None,
       name=Name(first='дмитрий',
                 last='медведев'))

Person(position=None,
       name=Name(first=None,
                 last='ниий'))

Person(position=None,
       name=Name(first='маттиас',
                 last='платцек'))

Person(position=None,
       name=Name(first=None,
                 last='боев'))

Person(position=None,
       name=Name(first='янис',
                 last='варуфакис'))

Person(position=None,
       name=Name(first=None,
                 last='зубков'))

Person(position=None,
       name=Name(first='владимир',
                 last='путин'))

Person(position=None,
       name=Name(first='дмитрий',
                 last='медведев'))

Person(position=None,
       name=Name(first='дмитрия',
                 last='песков'))

Person(position=None,
       name=Name(first='давид',
                 last='бакрадзе'))